# DeepSeek V3 Tool Use 测试与实现

这个 notebook 演示了如何使用 DeepSeek V3 模型实现 tool use 功能（原称为 function calling）。

In [2]:
import asyncio
import json
import os
from dotenv import load_dotenv
from fallback_openai_client import AsyncFallbackOpenAIClient

# 加载环境变量
load_dotenv()

# 配置 DeepSeek API
DEEPSEEK_API_KEY = os.getenv("API_KEY")
DEEPSEEK_BASE_URL = os.getenv("BASE_URL") 
DEEPSEEK_MODEL = "deepseek-v3-250324"

print(f"API Key: {DEEPSEEK_API_KEY[:20]}..." if DEEPSEEK_API_KEY else "API Key not found")
print(f"Base URL: {DEEPSEEK_BASE_URL}")
print(f"Model: {DEEPSEEK_MODEL}")

API Key: 7b448f0e-e6cb-421e-9...
Base URL: https://ark.cn-beijing.volces.com/api/v3
Model: deepseek-v3-250324


## 1. 测试基本 API 连接

首先测试 DeepSeek API 是否能正常调用。

In [3]:
async def test_basic_api():
    """测试基本的 API 连接"""
    client = AsyncFallbackOpenAIClient(
        primary_api_key=DEEPSEEK_API_KEY,
        primary_base_url=DEEPSEEK_BASE_URL,
        primary_model_name=DEEPSEEK_MODEL
    )
    
    try:
        response = await client.chat_completions_create(
            messages=[
                {"role": "user", "content": "你是谁？"}
            ],
            max_tokens=100,
            temperature=0.7
        )
        
        print("✅ API 连接成功!")
        print(f"响应: {response.choices[0].message.content}")
        return True
        
    except Exception as e:
        print(f"❌ API 连接失败: {e}")
        return False
    
    finally:
        await client.close()

# 运行测试
await test_basic_api()

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
✅ API 连接成功!
响应: 我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！🤖✨ 我可以帮你解答各种问题，无论是学习、工作，还是日常生活中的小困惑，都可以来找我聊聊！有什么我可以帮你的吗？😊


True

看一下模型的返回结果是什么样子的。

In [4]:
client = AsyncFallbackOpenAIClient(
        primary_api_key=DEEPSEEK_API_KEY,
        primary_base_url=DEEPSEEK_BASE_URL,
        primary_model_name=DEEPSEEK_MODEL
    )

response = await client.chat_completions_create(
    messages=[
        {"role": "user", "content": "你是谁？"}
    ],
    max_tokens=100,
    temperature=0.7
)

response

⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。


ChatCompletion(id='021748325529967917772c69bfe765b48361086187c9fae2a9874', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！✨ 我的使命是帮助你解答问题、提供信息、陪你聊天，甚至帮你处理各种文本和文件。无论是学习、工作，还是日常生活中的疑问，都可以来找我聊聊！😊  \n\n有什么我可以帮你的吗？', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1748325533, model='deepseek-v3-250324', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=68, prompt_tokens=5, total_tokens=73, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [ ]:

# 获取模型的响应内容 
response.choices[0].message.content

'我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！✨ 我的使命是帮助你解答问题、提供信息、陪你聊天，甚至帮你处理各种文本和文件。无论是学习、工作，还是日常生活中的疑问，都可以来找我聊聊！😊  \n\n有什么我可以帮你的吗？'

我们可以看到模型输出的内容是自身带有决定调用的工具的，function_call同理。

In [ ]:
response.choices[0].message
# ChatCompletionMessage(content='我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！✨ 我的使命是帮助你解答问题、提供信息、陪你聊天，甚至帮你处理各种文本和文件。无论是学习、工作，还是日常生活中的疑问，都可以来找我聊聊！😊  \n\n有什么我可以帮你的吗？', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


ChatCompletionMessage(content='我是DeepSeek Chat，由深度求索公司（DeepSeek）创造的智能AI助手！✨ 我的使命是帮助你解答问题、提供信息、陪你聊天，甚至帮你处理各种文本和文件。无论是学习、工作，还是日常生活中的疑问，都可以来找我聊聊！😊  \n\n有什么我可以帮你的吗？', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)

## 2. 定义 Tool Use 工具

定义一些示例函数供模型调用。

In [5]:
import math
from datetime import datetime

# 定义可供调用的函数
def get_current_time():
    """获取当前时间"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate_circle_area(radius: float):
    """计算圆的面积"""
    if radius <= 0:
        return {"error": "半径必须大于0"}
    return {"area": math.pi * radius ** 2}

def get_weather_info(city: str):
    """获取天气信息（模拟）"""
    weather_data = {
        "北京": "晴天，温度 15°C",
        "上海": "多云，温度 18°C", 
        "广州": "雨天，温度 22°C",
        "深圳": "晴天，温度 25°C"
    }
    return weather_data.get(city, f"抱歉，暂无 {city} 的天气信息")

def search_information(query: str):
    """搜索信息（模拟）"""
    search_results = {
        "Python": "Python 是一种高级编程语言，由 Guido van Rossum 创建。",
        "AI": "人工智能是计算机科学的一个分支，致力于创建能够模拟人类智能的系统。",
        "机器学习": "机器学习是人工智能的一个子集，通过数据训练算法来做出预测或决策。"
    }
    
    for key, value in search_results.items():
        if key.lower() in query.lower():
            return value
    
    return f"抱歉，没有找到关于 '{query}' 的相关信息。"

# 函数映射表
AVAILABLE_FUNCTIONS = {
    "get_current_time": get_current_time,
    "calculate_circle_area": calculate_circle_area,
    "get_weather_info": get_weather_info,
    "search_information": search_information
}

print("✅ 函数定义完成")

✅ 函数定义完成


## 3. Tool Use 工具定义

为 OpenAI Tools API 格式定义工具规范。

In [ ]:
# 定义工具规范（OpenAI Tools API 格式）
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "获取当前的日期和时间",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "calculate_circle_area",
            "description": "根据半径计算圆的面积",
            "parameters": {
                "type": "object",
                "properties": {
                    "radius": {
                        "type": "number",
                        "description": "圆的半径，必须大于0"
                    }
                },
                "required": ["radius"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_info", 
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称，如：北京、上海、广州、深圳"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_information",
            "description": "搜索相关信息",
            "parameters": {
                "type": "object", 
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

print(f"✅ 定义了 {len(TOOLS)} 个工具")
for tool in TOOLS:
    print(f"  - {tool['function']['name']}: {tool['function']['description']}")

✅ 定义了 4 个工具
  - get_current_time: 获取当前的日期和时间
  - calculate_circle_area: 根据半径计算圆的面积
  - get_weather_info: 获取指定城市的天气信息
  - search_information: 搜索相关信息


## 4. Tool Use 核心实现

实现 tool use 的核心逻辑。

In [ ]:
class DeepSeekToolCaller:
    """DeepSeek Tool Use 实现类（原 Function Call）"""
    
    def __init__(self, api_key: str, base_url: str, model: str):
        self.client = AsyncFallbackOpenAIClient(
            primary_api_key=api_key,
            primary_base_url=base_url,
            primary_model_name=model
        )
        self.tools = TOOLS
        self.available_functions = AVAILABLE_FUNCTIONS
    
    async def execute_tool_call(self, function_name: str, arguments: dict):
        """执行工具调用"""
        try:
            if function_name not in self.available_functions:
                return {"error": f"工具 {function_name} 不存在"}
            
            func = self.available_functions[function_name]
            
            # 执行函数
            if arguments:
                result = func(**arguments)
            else:
                result = func()
            
            return {"success": True, "result": result}
            
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    async def chat_with_tools(self, messages: list, max_tool_calls: int = 5):
        """支持 tool use 的对话"""
        conversation_messages = messages.copy()
        tool_call_count = 0
        
        while tool_call_count < max_tool_calls:
            try:
                # 调用 API
                response = await self.client.chat_completions_create(
                    messages=conversation_messages,
                    tools=self.tools,
                    tool_choice="auto",
                    temperature=0.7,
                    max_tokens=1000
                )
                
                message = response.choices[0].message
                conversation_messages.append({
                    "role": "assistant",
                    "content": message.content,
                    "tool_calls": message.tool_calls
                })
                
                # 检查是否有 tool calls
                if message.tool_calls:
                    print(f"🔧 检测到 {len(message.tool_calls)} 个工具调用")
                    
                    # 处理每个 tool call
                    for tool_call in message.tool_calls:
                        function_name = tool_call.function.name
                        function_args = json.loads(tool_call.function.arguments)
                        
                        print(f"📞 调用工具: {function_name}")
                        print(f"📝 参数: {function_args}")
                        
                        # 执行工具
                        tool_result = await self.execute_tool_call(function_name, function_args)
                        
                        # 添加工具执行结果到对话
                        conversation_messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": json.dumps(tool_result, ensure_ascii=False)
                        })
                        
                        print(f"✅ 工具执行结果: {tool_result}")
                    
                    tool_call_count += 1
                    continue  # 继续下一轮对话
                
                else:
                    # 没有 tool calls，返回最终响应
                    return {
                        "response": message.content,
                        "conversation": conversation_messages,
                        "tool_calls_used": tool_call_count
                    }
                    
            except Exception as e:
                return {
                    "error": f"API 调用失败: {e}",
                    "conversation": conversation_messages,
                    "tool_calls_used": tool_call_count
                }
        
        # 达到最大工具调用次数
        return {
            "response": "已达到最大工具调用次数限制",
            "conversation": conversation_messages,
            "tool_calls_used": tool_call_count
        }
    
    async def close(self):
        """关闭客户端"""
        await self.client.close()

print("✅ DeepSeekToolCaller 类定义完成")

✅ DeepSeekFunctionCaller 类定义完成


## 5. 测试 Tool Use 功能

测试各种 tool use 场景。

In [ ]:
async def test_tool_calls():
    """测试 tool use 功能"""
    
    caller = DeepSeekToolCaller(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL, 
        model=DEEPSEEK_MODEL
    )
    
    test_cases = [
        "现在几点了？",
        "帮我计算半径为5的圆的面积",
        "北京今天天气怎么样？",
        "搜索一下关于Python的信息",
        "现在时间是多少，然后告诉我广州的天气情况"
    ]
    
    try:
        for i, question in enumerate(test_cases, 1):
            print(f"\n{'='*60}")
            print(f"测试用例 {i}: {question}")
            print('='*60)
            
            messages = [{"role": "user", "content": question}]
            
            result = await caller.chat_with_tools(messages)
            
            if "error" in result:
                print(f"❌ 错误: {result['error']}")
            else:
                print(f"🤖 AI 响应: {result['response']}")
                print(f"📊 使用了 {result['tool_calls_used']} 次工具调用")
            
            # 添加分隔符
            print("-" * 60)
    
    finally:
        await caller.close()

# 运行测试
await test_tool_calls()

## 6. 交互式对话测试

创建一个简单的交互式对话示例。

In [ ]:
async def interactive_chat_demo():
    """交互式对话演示"""
    
    caller = DeepSeekToolCaller(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL,
        model=DEEPSEEK_MODEL
    )
    
    # 预设一些演示问题
    demo_questions = [
        "你好！你可以帮我做什么？",
        "现在几点了？顺便告诉我深圳的天气",
        "计算一下半径为10的圆的面积",
        "搜索关于机器学习的信息"
    ]
    
    conversation_history = []
    
    try:
        print("🚀 开始交互式对话演示")
        print("📱 可用功能：获取时间、计算圆面积、查询天气、搜索信息")
        print("=" * 60)
        
        for question in demo_questions:
            print(f"\n👤 用户: {question}")
            
            # 添加用户消息到历史
            conversation_history.append({"role": "user", "content": question})
            
            # 获取 AI 响应
            result = await caller.chat_with_tools(conversation_history.copy())
            
            if "error" in result:
                print(f"❌ 错误: {result['error']}")
            else:
                print(f"🤖 助手: {result['response']}")
                
                # 更新对话历史（只保留用户和助手的主要对话）
                conversation_history.append({
                    "role": "assistant", 
                    "content": result['response']
                })
                
                if result['tool_calls_used'] > 0:
                    print(f"🔧 (使用了 {result['tool_calls_used']} 次工具调用)")
    
    finally:
        await caller.close()
        print(f"\n✅ 演示结束")

# 运行交互式演示
await interactive_chat_demo()

## 7. 高级功能：批量处理

演示批量处理多个请求的功能。

In [ ]:
async def batch_processing_demo():
    """批量处理演示"""
    
    caller = DeepSeekToolCaller(
        api_key=DEEPSEEK_API_KEY,
        base_url=DEEPSEEK_BASE_URL,
        model=DEEPSEEK_MODEL
    )
    
    # 批量任务
    batch_tasks = [
        "告诉我现在的时间",
        "计算半径为3的圆的面积", 
        "上海的天气如何？",
        "搜索Python编程相关信息",
        "计算半径为7的圆的面积，然后告诉我北京天气"
    ]
    
    print("🔄 开始批量处理演示")
    print(f"📝 总共 {len(batch_tasks)} 个任务")
    print("=" * 60)
    
    results = []
    
    try:
        for i, task in enumerate(batch_tasks, 1):
            print(f"\n📋 任务 {i}/{len(batch_tasks)}: {task}")
            
            messages = [{"role": "user", "content": task}]
            result = await caller.chat_with_tools(messages)
            
            if "error" in result:
                print(f"❌ 失败: {result['error']}")
                results.append({"task": task, "success": False, "error": result['error']})
            else:
                print(f"✅ 完成: {result['response']}")
                results.append({
                    "task": task, 
                    "success": True, 
                    "response": result['response'],
                    "tool_calls": result['tool_calls_used']
                })
        
        # 总结
        print(f"\n📊 批量处理总结:")
        print(f"✅ 成功: {sum(1 for r in results if r['success'])} 个")
        print(f"❌ 失败: {sum(1 for r in results if not r['success'])} 个")
        print(f"🔧 总工具调用: {sum(r.get('tool_calls', 0) for r in results)} 次")
        
    finally:
        await caller.close()

# 运行批量处理演示
await batch_processing_demo()

## 8. 总结

这个 notebook 演示了如何使用 DeepSeek V3 模型实现 tool use 功能，包括：

1. ✅ **基本 API 连接测试** - 验证 API 配置正确性
2. 🛠️ **工具定义** - 创建可供模型调用的工具函数
3. 📋 **工具规范** - 按 OpenAI Tools API 格式定义函数规范
4. 🤖 **核心实现** - DeepSeekToolCaller 类
5. 🧪 **功能测试** - 测试各种 tool use 场景
6. 💬 **交互式对话** - 演示连续对话中的工具调用
7. 📦 **批量处理** - 演示批量任务处理能力

### 主要特性:
- 🔄 自动检测和执行工具调用
- 🛡️ 错误处理和重试机制
- 📝 完整的对话历史管理
- 🎯 支持多轮对话中的工具调用
- 📊 详细的执行统计信息

### Tool Use vs Function Calling:
OpenAI API 现在使用 "tools" 和 "tool_calls" 术语替代了之前的 "function calling"，这反映了功能的扩展：
- 🔧 **Tools**: 包含各种类型的工具（函数、代码解释器等）
- 📞 **Tool Calls**: 模型请求调用特定工具的操作
- 🔄 **Tool Choice**: 控制模型如何选择使用工具

### 可扩展的功能:
- 添加更多自定义工具
- 实现工具调用缓存
- 添加工具调用权限控制
- 集成外部 API 服务
- 支持更多工具类型（如代码解释器）